In [3]:
# !pip install tsgm

In [4]:
import tensorflow as tf
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, LayerNormalization, Dense, Attention, MultiHeadAttention, Lambda
from tensorflow.keras.models import Model, Sequential
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras import backend as K
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from keras.layers import Flatten
from keras.layers import Dropout
# import seaborn as sns
# import tsgm

In [5]:
# !gdown --fuzzy https://drive.google.com/file/d/1F9uinZY-eG4x9dNsUOOtAAZMYq6p945U/view?usp=drive_link
# !unzip -qq "ASL-Sensor-Dataglove-Dataset.zip" -d glove_data
# !echo "Unzip successfully"

In [6]:
class TimeSeriesDataset:
    def __init__(self, root_dir, feature_names=[]):
        self.data = self.load_data(root_dir, feature_names)
        self.merge_flex_sensors()

    def load_data(self, root_dir, feature_names):
        data = []

        for individual_dir in sorted(os.listdir(root_dir)):
            individual_path = os.path.join(root_dir, individual_dir)
            for class_dir in sorted(os.listdir(individual_path)):
                class_path = os.path.join(individual_path, class_dir)
                if os.path.isdir(class_path):
                    for file in glob.glob(os.path.join(class_path, "*.csv")):
                        df = pd.read_csv(file, usecols=feature_names)
                        class_name = os.path.splitext(os.path.basename(file))[0]
                        df["class"] = class_name
                        data.append(df)


        # Concatenate all data frames into a single data frame
        data = pd.concat(data, ignore_index=True)
        return data
    
    def merge_flex_sensors(self):
        if 'flex_1' in self.data.columns and 'flex_2' in self.data.columns:
            # Define a small epsilon value to avoid division by zero
            epsilon = 1e-10

            # Convert flex sensor values to conductance (1/R), handling zero values
            conductance_4 = 1 / (self.data['flex_1'] + epsilon)
            conductance_5 = 1 / (self.data['flex_2'] + epsilon)
            
            # Sum the conductances
            total_conductance = conductance_4 + conductance_5
            
            # Convert back to resistance, handling very large values
            self.data['flex_1'] = np.where(
                total_conductance > epsilon,
                1 / total_conductance,
                np.finfo(float).max  # Use maximum float value for near-zero conductance
            )
            
            # Drop original columns
            self.data = self.data.drop(columns=['flex_2'])

In [7]:
root_dir = "glove_data/"
feature_names = [
    "flex_1", "flex_3", "flex_4", "flex_5",
    "GYRx", "GYRy", "GYRz"
]

dataset = TimeSeriesDataset(root_dir, feature_names).data
# dataset = dataset.sort_values(by=["class"])

# filter_classes = ["deaf", "fine", "good", "goodbye", "hello"]
# dataset = dataset[dataset["class"].isin(filter_classes)]

x_data, y_data = dataset.iloc[:, :-1].values, dataset.iloc[:, -1].values

scaler = StandardScaler()
# x_data = scaler.fit_transform(x_data)

label_encoder = LabelEncoder()
y_data = label_encoder.fit_transform(y_data)

timesteps = 150
n_features = 7
num_classes = len(np.unique(y_data))

num_samples = len(y_data) // timesteps

x_data = x_data[:num_samples * timesteps].reshape((num_samples, timesteps, n_features))
y_data = y_data[:num_samples * timesteps:timesteps]


In [8]:
from scipy import interpolate


# Define original and new timesteps
old_timesteps = np.arange(150)
new_timesteps = np.linspace(0, 149, 240)  # New time steps from 0 to 149, but 240 steps in between

# Initialize an empty array for the upsampled data
upsampled_data = np.zeros((x_data.shape[0], 240, x_data.shape[2]))

# Interpolate along the time dimension for each sample and each feature
for i in range(x_data.shape[0]):  # Iterate over each sample
    for j in range(x_data.shape[2]):  # Iterate over each feature
        f = interpolate.interp1d(old_timesteps, x_data[i, :, j], kind='linear')
        upsampled_data[i, :, j] = f(new_timesteps)

print(upsampled_data.shape)  # Should print (10000, 240, 7)

(10000, 240, 7)


In [9]:
temp = pd.DataFrame(upsampled_data.reshape(-1, n_features))
print(temp.head().to_markdown())

|    |       0 |       1 |       2 |       3 |          4 |          5 |         6 |
|---:|--------:|--------:|--------:|--------:|-----------:|-----------:|----------:|
|  0 | 20      | 72      | 77      | 58      | -0.068702  | -0.015267  | 0.015267  |
|  1 | 20      | 70.7531 | 76.3766 | 58      | -0.0734613 | -0.015267  | 0.0200263 |
|  2 | 20      | 70.7406 | 75.7531 | 58.4937 | -0.076336  | -0.0171515 | 0.022901  |
|  3 | 20      | 72.6109 | 75.1297 | 59.7406 | -0.076336  | -0.0219108 | 0.022901  |
|  4 | 19.5063 | 73      | 76.9749 | 59.0126 | -0.076336  | -0.022901  | 0.022901  |


In [10]:
x_data = upsampled_data
timesteps = 240

In [11]:
# aug_model = tsgm.models.augmentations.GaussianNoise()
# x_data_aug = aug_model.generate(x_data, n_samples=x_data.shape[0], variance=0.2)

# x_data = np.concatenate((x_data, x_data_aug), axis=0)
# y_data = np.concatenate((y_data, y_data), axis=0)


In [12]:
print(x_data.shape)
print(y_data.shape)

(10000, 240, 7)
(10000,)


In [13]:
# def positional_encoding(length, depth):
#     depth = int(depth)
#     positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
#     depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

#     angle_rates = 1 / (10000**depths)                # (1, depth)
#     angle_rads = positions * angle_rates             # (pos, depth)

#     pos_encoding = np.concatenate(
#         [np.sin(angle_rads), np.cos(angle_rads)],
#         axis=-1)

#     return tf.cast(pos_encoding, dtype=tf.float32)

# class AddPositionalEncoding(tf.keras.layers.Layer):
#     def __init__(self):
#         super().__init__()

#     def build(self, input_shape):
#         _, seq_len, d_model = input_shape
#         self.pos_encoding = positional_encoding(seq_len, d_model)

#     def call(self, inputs):
#         # Ensure positional encoding has the same shape as the input
#         return inputs + self.pos_encoding[:tf.shape(inputs)[1], :tf.shape(inputs)[2]]

#     def compute_output_shape(self, input_shape):
#         return input_shape

#     def get_config(self):
#         config = super().get_config()
#         return config

# def create_model(timesteps, n_features, num_classes):
#     inputs = Input(shape=(timesteps, n_features))

#     x = Conv1D(filters=64, kernel_size=5, activation='relu')(inputs)
#     x = MaxPooling1D(pool_size=2)(x)

#     x = Conv1D(filters=64, kernel_size=5, activation='relu')(x)
#     x = MaxPooling1D(pool_size=2)(x)

#     x = Conv1D(filters=64, kernel_size=5, activation='relu')(x)
#     x = MaxPooling1D(pool_size=2)(x)

#     x = Conv1D(filters=64, kernel_size=5, activation='relu')(x)
#     x = MaxPooling1D(pool_size=2)(x)

#     x = LSTM(units=128, return_sequences=True)(x)
#     x = LSTM(units=128, return_sequences=True)(x)

#     x = AddPositionalEncoding()(x)

#     # MultiHeadAttention layer
#     attn_output = MultiHeadAttention(num_heads=4, key_dim=128)(x, x, x)
#     x = LayerNormalization()(attn_output + x)

#     x = Dense(units=128, activation='relu')(x)

#     # Global Attention layer
#     attn = Attention()([x, x])
#     x = LayerNormalization()(attn + x)

#     # Global average pooling to reduce sequence dimension
#     x = tf.keras.layers.GlobalAveragePooling1D()(x)

#     outputs = Dense(num_classes, activation='softmax')(x)

#     model = Model(inputs=inputs, outputs=outputs)
#     return model

def create_model(timesteps, features, num_classes, name = "original_model"):
    # define model
    model = Sequential(name=name)
    model.add(LSTM(units = 128, input_shape = (timesteps, features)))
    model.add(Dropout(0.5)) 
    model.add(Flatten())
    model.add(Dense(units = 64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [14]:
model = create_model(timesteps, n_features, num_classes)
model.summary()

# checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model_weights.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_data, y_data, epochs=30, batch_size=64, validation_split = 0.2, callbacks=[early_stopping])

c:\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "original_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        69,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 40)             │         2,600 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,488 (314.41 KB)

 Trainable params: 80,488 (314.41 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 155ms/step - accuracy: 0.1097 - loss: 3.4050 - val_accuracy: 0.1960 - val_loss: 2.5437
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 155ms/step - accuracy: 0.2678 - loss: 2.2539 - val_accuracy: 0.2495 - val_loss: 2.2310
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 160ms/step - accuracy: 0.3195 - loss: 1.9669 - val_accuracy: 0.2815 - val_loss: 2.0829
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 151ms/step - accuracy: 0.3547 - loss: 1.8274 - val_accuracy: 0.2985 - val_loss: 2.0024
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 153ms/step - accuracy: 0.3617 - loss: 1.7581 - val_accuracy: 0.3030 - val_loss: 1.9402
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 151ms/step - accuracy: 0.3783 - loss: 1.7399 - val_accuracy: 0.3300 - val_loss: 1.9485
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 152ms/step - accuracy: 0.3881 - loss: 1.6988 - val_accuracy: 0.3350 - val_loss: 1.8994
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 157ms/step - accuracy: 0.4020 - loss: 1

In [15]:
if not os.path.exists("model"):
    os.makedirs("model")
model.save("model/LSTM_TransferLearning.h5")

In [20]:
# y_pred = model.predict(x_test)
# y_pred = np.argmax(y_pred, axis=1)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

In [17]:
# cm = confusion_matrix(y_test, y_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(label_encoder.classes_))
# disp.plot(cmap=plt.cm.Blues)
# plt.show()

In [18]:
# print(classification_report(y_test, y_pred))